# **Methylation Biomarkers for Predicting Cancer**

## **Data Pre-Processing**

**Author:** Meg Hutch

**Date:** February 25, 2020

**Objective:** Pre-process data for use in Neural Networks, Randfom Forest, and logistic regression.

**Note:** In this version, I will only test the ability of methylation levels to classify cancer types. I will not include phenotypic data for now. Additionally, this version has our data split 70% for training and 30% for testing. The 70% training data will undergo leave-one-out-cross-fold validation to tune hyperparameters prior to testing final performance on the 30% test set. 

Note: This is the new version of the script where we normalize gene counts using DEseq2 in the initial pre-processing script in R. This provided more than double the number of Principal Components that make up 90% of the variance (157). Regardless, we will begin running the deep learning classifier on the revised data. 




**Notes/ToDos: Will need to similarly normalize/modify the final testing sets as well. For now, I just focused on the various training sets. I also have sections to create a few more datasets, combining stomach/colon cancer + removing GBM + BC, will work on this after initital analyses**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors
import seaborn as sns

In [ ]:
# set working directory for git hub
import os
#os.chdir('/projects/p31049/Multi_Cancer_DL/')
os.chdir('C:\\Users\\User\\Box Sync/Projects/Multi_Cancer_DL/')
os. getcwd()

**Import Training, Testing, and Principal component data**

In [ ]:
# Training set
mcTrain = pd.read_csv('02_Processed_Data/mcTrain_70_30.csv')
# Testing set
mcTest = pd.read_csv('02_Processed_Data/mcTest_70_30.csv')

In [ ]:
mcTrain.head()

# **Pre-Process Data**

In [ ]:
# remove genetic data from the mcTrain dataset
mcTrain = mcTrain[['seq_num','diagnosis', 'dilute_library_concentration', 'age', 'gender', 'frag_mean']]

# do the same for the testing set
mcTest = mcTest[['seq_num','diagnosis', 'dilute_library_concentration', 'age', 'gender', 'frag_mean']]

In [ ]:
mcTrain

**Shuffle the training and test sets**

Currently, all disease states are in order - we don't want to feed to the network in order!

In [ ]:
import random
random.seed(222020)
mcTrain = mcTrain.sample(frac=1, axis = 0).reset_index(drop=True) # frac = 1 returns all rows in random order
mcTest = mcTest.sample(frac=1, axis = 0).reset_index(drop=True)

**Create a new numeric index and drop seq_num and demographic data for these experiments**

For future code we want the index to be numeric

In [ ]:
# Create new ids
mcTrain['id'] = mcTrain.index + 1
mcTest['id'] = mcTest.index + 243

# Drop num_seq
mcTrain = mcTrain.drop(columns=["seq_num", "dilute_library_concentration", "age", "gender", "frag_mean"])
mcTest = mcTest.drop(columns=["seq_num", "dilute_library_concentration", "age", "gender", "frag_mean"])

**Remove Labels (Diagnosis) from the datasets**

In [ ]:
mcTrain_x = mcTrain.drop(columns=["diagnosis"])
mcTest_x = mcTest.drop(columns=["diagnosis"])

**Create Labeled Datasets**

In [ ]:
mcTrain_y = mcTrain[['id','diagnosis']]
mcTest_y = mcTest[['id','diagnosis']]

In [ ]:
# Examine the unique target variables
mcTrain_y.diagnosis.unique()

In [ ]:
# Replace each outcome target with numerical value
mcTrain_y = mcTrain_y.replace('HEA', 0)
mcTrain_y = mcTrain_y.replace('CRC', 1)
mcTrain_y = mcTrain_y.replace('ESCA', 2)
mcTrain_y = mcTrain_y.replace('HCC', 3)
mcTrain_y = mcTrain_y.replace('STAD', 4)
mcTrain_y = mcTrain_y.replace('GBM', 5)
mcTrain_y = mcTrain_y.replace('BRCA', 6)

mcTest_y = mcTest_y.replace('HEA', 0)
mcTest_y = mcTest_y.replace('CRC', 1)
mcTest_y = mcTest_y.replace('ESCA', 2)
mcTest_y = mcTest_y.replace('HCC', 3)
mcTest_y = mcTest_y.replace('STAD', 4)
mcTest_y = mcTest_y.replace('GBM', 5)
mcTest_y = mcTest_y.replace('BRCA', 6)

**Convert seq_num id to index**

In [ ]:
mcTrain_x = mcTrain_x.set_index('id')
mcTrain_y = mcTrain_y.set_index('id')

mcTest_x = mcTest_x.set_index('id')
mcTest_y = mcTest_y.set_index('id')

# **Save the Main Training and Testing Datasets**

In [ ]:
mcTrain_x.to_csv('02_Processed_Data/Final_Datasets/mcTrain_x_Full_70_30.csv')
mcTrain_y.to_csv('02_Processed_Data/Final_Datasets/mcTrain_y_Full_70_30.csv')

# **Downsampling the Majority Class**

In [ ]:
# Subset healthy patients
class0 = mcTrain_y[mcTrain_y.diagnosis == 0]

# Select only 30 healthy subjects - we will remove these subjects
class0 = class0.head(30)
class0 = class0.index.tolist()

# remove these patients from the main dataframe 
mcTrain_y_ds = mcTrain_y[~mcTrain_y.index.isin(class0)]
#print(mcTrain_y.head(20))

# Print the cases we wanted to remove
#print(class0)

#Observe class distributions
class0_new = mcTrain_y[mcTrain_y.diagnosis == 0]
#print('# Healthy Subjects', class0_new.shape) # shoud be 31
#print('# Full Training Set', mcTrain_y.shape)

**Remove excess healthy patients from the input training set and original dataset**

In [ ]:
mcTrain_x_ds = mcTrain_x[~mcTrain_x.index.isin(class0)]
#print('# Full Training Set', mcTrain_x)

# **Save the Downsampled Training Data**

In [ ]:
mcTrain_x_ds.to_csv('02_Processed_Data/Final_Datasets/mcTrain_x_ds_70_30.csv')
mcTrain_y_ds.to_csv('02_Processed_Data/Final_Datasets/mcTrain_y_ds_70_30.csv')

# **Equalize All Classes**

In [ ]:
# Subset healthy patients
class0 = mcTrain_y[mcTrain_y.diagnosis == 0]
class1 = mcTrain_y[mcTrain_y.diagnosis == 1]
class2 = mcTrain_y[mcTrain_y.diagnosis == 2]
class3 = mcTrain_y[mcTrain_y.diagnosis == 3]
class4 = mcTrain_y[mcTrain_y.diagnosis == 4]
class5 = mcTrain_y[mcTrain_y.diagnosis == 5]
class6 = mcTrain_y[mcTrain_y.diagnosis == 6]


# Select only top 20 patients - we will remove these patients
class0 = class0.head(20)
class0 = class0.index.tolist()

class1 = class1.head(20)
class1 = class1.index.tolist()

class2 = class2.head(20)
class2 = class2.index.tolist()

class3 = class3.head(20)
class3 = class3.index.tolist()

class4 = class4.head(20)
class4 = class4.index.tolist()

class5 = class5.head(20)
class5 = class5.index.tolist()

class6 = class6.head(20)
class6 = class6.index.tolist()

# Subset the main mcTrain database with the 20 patients in each class (note: gbm and breast cancer will have 19 and 18)
mcTrain_y_ds_20 = mcTrain_y[mcTrain_y.index.isin(class0)]
mcTrain_y_ds_20_1 = mcTrain_y[mcTrain_y.index.isin(class1)]
mcTrain_y_ds_20_2 = mcTrain_y[mcTrain_y.index.isin(class2)]
mcTrain_y_ds_20_3 = mcTrain_y[mcTrain_y.index.isin(class3)]
mcTrain_y_ds_20_4 = mcTrain_y[mcTrain_y.index.isin(class4)]
mcTrain_y_ds_20_5 = mcTrain_y[mcTrain_y.index.isin(class5)]
mcTrain_y_ds_20_6 = mcTrain_y[mcTrain_y.index.isin(class6)]

# bind all dataframes
mcTrain_y_ds_20 = mcTrain_y_ds_20.append(pd.DataFrame(data = mcTrain_y_ds_20_1))
mcTrain_y_ds_20 = mcTrain_y_ds_20.append(pd.DataFrame(data = mcTrain_y_ds_20_2))
mcTrain_y_ds_20 = mcTrain_y_ds_20.append(pd.DataFrame(data = mcTrain_y_ds_20_3))
mcTrain_y_ds_20 = mcTrain_y_ds_20.append(pd.DataFrame(data = mcTrain_y_ds_20_4))
mcTrain_y_ds_20 = mcTrain_y_ds_20.append(pd.DataFrame(data = mcTrain_y_ds_20_5))
mcTrain_y_ds_20 = mcTrain_y_ds_20.append(pd.DataFrame(data = mcTrain_y_ds_20_6))

In [ ]:
mcTrain_y_ds_20.shape

**Modify the feature training x set**

In [ ]:
mcTrain_x_ds_20 = mcTrain_x[mcTrain_x.index.isin(class0)]
mcTrain_x_ds_20_1 = mcTrain_x[mcTrain_x.index.isin(class1)]
mcTrain_x_ds_20_2 = mcTrain_x[mcTrain_x.index.isin(class2)]
mcTrain_x_ds_20_3 = mcTrain_x[mcTrain_x.index.isin(class3)]
mcTrain_x_ds_20_4 = mcTrain_x[mcTrain_x.index.isin(class4)]
mcTrain_x_ds_20_5 = mcTrain_x[mcTrain_x.index.isin(class5)]
mcTrain_x_ds_20_6 = mcTrain_x[mcTrain_x.index.isin(class6)]

# bind all dataframes
mcTrain_x_ds_20 = mcTrain_x_ds_20.append(pd.DataFrame(data = mcTrain_x_ds_20_1))
mcTrain_x_ds_20 = mcTrain_x_ds_20.append(pd.DataFrame(data = mcTrain_x_ds_20_2))
mcTrain_x_ds_20 = mcTrain_x_ds_20.append(pd.DataFrame(data = mcTrain_x_ds_20_3))
mcTrain_x_ds_20 = mcTrain_x_ds_20.append(pd.DataFrame(data = mcTrain_x_ds_20_4))
mcTrain_x_ds_20 = mcTrain_x_ds_20.append(pd.DataFrame(data = mcTrain_x_ds_20_5))
mcTrain_x_ds_20 = mcTrain_x_ds_20.append(pd.DataFrame(data = mcTrain_x_ds_20_6))

In [ ]:
mcTrain_x_ds_20.shape

# **Save Equal Downsampled Datasets** 

In [ ]:
mcTrain_x_ds.to_csv('02_Processed_Data/Final_Datasets/mcTrain_x_ds_70_30.csv')
mcTrain_y_ds.to_csv('02_Processed_Data/Final_Datasets/mcTrain_y_ds_70_30.csv')

# **Combine Stomach + Colon Cancer Patients**

# **Remove Breast Cancer + GBM Patients**